In [14]:
import os
from dotenv import load_dotenv

In [15]:
# Load environment variables from a .env file
load_dotenv()

video_url = os.getenv('video_url')
print(video_url)

https://www.youtube.com/watch?v=5qap5aO4i9A


In [ ]:
from googleapiclient.discovery import build

# Replace with your actual API key
API_KEY = os.getenv('DEVELOPER_KEY')

# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

def get_channel_videos(channel_id):
    videos = []
    next_page_token = None
    
    while True:
        # Request channel uploads playlist
        res = youtube.channels().list(id=channel_id, 
                                      part='contentDetails').execute()
        playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        
        # Request playlist items
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                           part='snippet',
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        
        # Process video items
        for item in res['items']:
            video_id = item['snippet']['resourceId']['videoId']
            title = item['snippet']['title']
            videos.append({'id': video_id, 'title': title})
        
        # Check for next page
        next_page_token = res.get('nextPageToken')
        if not next_page_token:
            break
    
    return videos

# Replace with the desired channel ID
channel_id = 'UCqAEtEr0A0Eo2IVcuWBfB9g'

video_list = get_channel_videos(channel_id)

# Print the list of videos
for video in video_list:
    print(f"Video ID: {video['id']}, Title: {video['title']}")
